<a href="https://colab.research.google.com/github/decastro-alex/fisher-black/blob/main/problem_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install yfinance pandas matplotlib plotly

In [10]:
import yfinance as yf
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime

# Define the stock tickers for gold mining companies and the market index (SPY ETF)
gold_mining_companies = ['GOLD', 'NEM', 'FNV']
market_index = 'SPY'

# Define the time period
start_date = '1900-01-01'  # As far back as possible
end_date = datetime.today().strftime('%Y-%m-%d')  # Today's date

# Fetch historical data
data = yf.download(gold_mining_companies + [market_index], start=start_date, end=end_date)['Adj Close']

# Calculate daily returns
daily_returns = data.pct_change().dropna()

# Rolling windows in trading days (approximations: 1 month = 21 days, 3 months = 63 days, 6 months = 126 days, 12 months = 252 days)
rolling_windows = [21, 63, 126, 252]

# Color scheme
colors = {'GOLD': 'gold', 'NEM': 'green', 'FNV': 'blue'}

# Loop through each rolling window to create individual plots
for window in rolling_windows:
    # Calculate rolling covariance with the market index (SPY)
    rolling_cov = daily_returns.rolling(window=window).cov(daily_returns[market_index]).drop(market_index, axis=1)

    # Calculate rolling variance for the market index (SPY)
    rolling_var = daily_returns[market_index].rolling(window=window).var()

    # Calculate rolling beta
    rolling_beta = rolling_cov.div(rolling_var, axis=0)

    # Smooth the plot using a simple moving average with a window of 5 days
    rolling_beta_smoothed = rolling_beta.rolling(window=5).mean()

    # Create the plot
    fig = go.Figure()

    for company, color in colors.items():
        fig.add_trace(go.Scatter(x=rolling_beta_smoothed.index, y=rolling_beta_smoothed[company],
                                 name=f"{company}", line=dict(color=color)))

    fig.update_layout(
        title=f'Rolling Beta ({window} days) with SPY',
        xaxis_title='Date',
        yaxis_title='Beta',
        height=600,
        width=800
    )

    fig.show()

[*********************100%%**********************]  4 of 4 completed


In [ ]:
3